In [20]:
import ipywidgets as w

In [31]:
intro = w.VBox(children=(w.HTML(value='<font size="+2"><b>CARA</b> Covid Calculator</font>'),
                         w.Text(placeholder="Simulation name"),
                         w.Text(placeholder="Room number")))

intro

Text(value='test', placeholder='Simulation name')
